<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Загрузите и подготовьте данные. Поясните порядок действий.
Исследуйте баланс классов, обучите модель без учёта дисбаланса. Кратко опишите выводы.
Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую. Кратко опишите выводы.
Проведите финальное тестирование.

In [1]:
# Импортируем все необходимое

import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.utils import shuffle

In [2]:
# Создаем исходный датафрейм

data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
# Посмотрим на информацию о датасете
data.info()
# 14 столбцов и 10000 строк

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
# Все числовые столбцы выглядят адекватно, выбросов похоже нет
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Признаки:
<ol>RowNumber — индекс строки в данных</ol>
<ol>CustomerId — уникальный идентификатор клиента</ol>
<ol>Surname — фамилия</ol>
<ol>CreditScore — кредитный рейтинг</ol>
<ol>Geography — страна проживания</ol>
<ol>Gender — пол</ol>
<ol>Age — возраст</ol>
<ol>Tenure — сколько лет человек является клиентом банка</ol>
<ol>Balance — баланс на счёте</ol>
<ol>NumOfProducts — количество продуктов банка, используемых клиентом</ol>
<ol>HasCrCard — наличие кредитной карты</ol>
<ol>IsActiveMember — активность клиента</ol>
<ol>EstimatedSalary — предполагаемая зарплата</ol>
<ol>Exited — факт ухода клиента</ol>

In [5]:
# В данных имеются пропуски по столбцу Tenure - сколько лет человек является клиентом банка
data['Tenure'].isnull().sum()
# 909 строк - удалим их, что бы они не мешали моделям. Оставшихся строк хватит для моделей
data = data.drop(data[data['Tenure'].isnull()].index)

In [6]:
# Проверим на дисбаланс классов
sum(data["Exited"]/len(data))
# Положительных значений всего 20%

0.2039379606204001

In [7]:
# Удалим лишние столбцы, которые не пригодятся для модели
data.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [8]:
# Применим технику OHE для столбцов с гендером и страной клиента
data = pd.get_dummies(data, drop_first=True)

In [9]:
# разделяем тестовые данные на признаки и результативную переменную
features = data.drop(['Exited'], axis=1)
target = data['Exited']

In [10]:
# Разделим выборки на тестовую, обучающую и валидационную
features_train, features_1, target_train, target_1 = train_test_split(features, target, 
                                                                              test_size=0.25, 
                                                                              random_state=42)

In [11]:
features_valid, features_test, target_valid, target_test = train_test_split(features_1, target_1, 
                                                                              test_size=0.5, 
                                                                              random_state=42)

In [12]:
# Проверим разделение на адекватность
features_train.shape, features_valid.shape, features_test.shape

((6818, 11), (1136, 11), (1137, 11))

In [13]:
features_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7749,785,38,1.0,0.00,1,1,0,134964.85,0,0,0
3308,813,62,10.0,64667.95,2,0,1,140454.14,0,0,0
3406,632,29,7.0,80922.75,1,1,0,7820.78,0,1,0
1364,850,39,3.0,124548.99,2,1,1,120380.12,1,0,1
9929,755,38,4.0,111096.91,1,1,1,19762.88,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
6312,658,28,9.0,152812.58,1,1,0,166682.57,1,0,0
5722,681,34,3.0,0.00,2,0,0,55816.20,0,0,1
5940,849,41,6.0,0.00,2,1,1,169203.51,0,0,0
966,563,34,6.0,139810.34,1,1,1,152417.79,0,0,0


In [14]:
# Спрячем предупреждения
pd.options.mode.chained_assignment = None

In [15]:
# Приведем данные к одному масштабу
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance','NumOfProducts','EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
pd.options.mode.chained_assignment = None

## Исследование задачи

In [16]:
# Обучим модели
# Сначала логистическая регрессия
model_lr = LogisticRegression()
model_lr.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
# Получаем небольшое значение F1-меры
predictions = model_lr.predict(features_valid)
print("Accuracy:", accuracy_score(predictions, target_valid))
print("F1:", f1_score(predictions, target_valid))

Accuracy: 0.7992957746478874
F1: 0.3090909090909091


In [18]:
# Добавим метрику площиди под кривой auc_roc
probabilities_valid = model_lr.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
print("Auc_roc:", auc_roc)

Auc_roc: 0.7673414341761117


In [19]:
# Посмотрим на лес
# Создаем модель
model_rfc = RandomForestClassifier()
# Создаем словарь для перебора параметров
param_grid = { 
    'n_estimators': [10, 20, 30, 40, 50],
    'max_depth' : [5,7,9,11,13,15],
}

In [20]:
# Перебираем параметры
CV_rfc = GridSearchCV(estimator=model_rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(features_train, target_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [21]:
# Предсказываем на основании лучшей модели
# Получаем результат немного лучше, чем у регрессии
predictions = CV_rfc.best_estimator_.predict(features_valid)
print("accuracy:", accuracy_score(predictions, target_valid))
print("F1:", f1_score(predictions, target_valid))

accuracy: 0.8389084507042254
F1: 0.49863013698630126


In [22]:
# Добавим метрику площиди под кривой auc_roc
probabilities_valid = CV_rfc.best_estimator_.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
print("Auc_roc:", auc_roc)

Auc_roc: 0.8521732417901773


## Борьба с дисбалансом

In [23]:
# Будем бороться с дисбалансом техникой upsampling

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled


In [24]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [25]:
# Проверяем логистическую регрессию
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled,target_upsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))
print("Accuracy:", accuracy_score(predicted_valid, target_valid))

F1: 0.5114285714285715
Accuracy: 0.698943661971831


In [26]:
# Проверяем лес
model_rfc = RandomForestClassifier()
# Создаем словарь для перебора параметров
param_grid = { 
    'n_estimators': [10, 20, 30, 40, 50, 60],
    'max_depth' : [5,7,9,11,13,15,17],
    'criterion':['gini','entropy']
}

In [27]:
# Перебираем параметры
CV_rfc = GridSearchCV(estimator=model_rfc, param_grid=param_grid, cv=7)
CV_rfc.fit(features_train, target_train)

GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [28]:
CV_rfc.best_params_

{'criterion': 'gini', 'max_depth': 11, 'n_estimators': 40}

In [29]:
# Предсказываем на основании лучшей модели
# Получаем результат лучше, чем у регрессии
predictions = CV_rfc.best_estimator_.predict(features_upsampled)
print("accuracy:", accuracy_score(predictions, target_upsampled))
print("F1:", f1_score(predictions, target_upsampled))

accuracy: 0.8088315464292204
F1: 0.7699037620297463


# Добавила случайный лес через цикл и технику уменьшения выборки

In [30]:
# Случайный лес с помощью цикла
best_accuracy = 0
best_depth = 0
best_f1 = 0
for depth in range(1,15):
    for est in range(5,50,5):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=42)
        model.fit(features_upsampled,target_upsampled)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(predictions, target_valid)
        f1 = f1_score(predictions, target_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est,  "  Качество:", best_accuracy, 'F1:', best_f1 )

Глубина дерева: 8     Количество деревьев: 15   Качество: 0.8283450704225352 F1: 0.6408839779005525


In [31]:
#  Борьба с дисбалансом классов путем уменьшения выборки

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled



In [32]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.5)

In [33]:
# Проверяем логистическую регрессию
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled,target_downsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))
print("Accuracy:", accuracy_score(predicted_valid, target_valid))

F1: 0.4978165938864629
Accuracy: 0.7975352112676056


In [34]:
# Случайный лес с помощью цикла
best_accuracy = 0
best_depth = 0
best_f1 = 0
for depth in range(1,30):
    for est in range(5,160,5):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=42)
        model.fit(features_downsampled,target_downsampled)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(predictions, target_valid)
        f1 = f1_score(predictions, target_valid)
        if f1 > best_f1:
            best_f1 = f1
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est,  "  Качество:", best_accuracy, 'F1:', best_f1 )

# ПРИ ТАКОМ ПЕРЕБОРЕ ВСЕ РАВНО ПАРАМЕТРЫ 13 И 110 НЕ ПОКАЗЫВАЮТ НУЖНОГО РЕЗУЛЬТАТА НА ТЕСТОВОЙ ВЫБОРКЕ

# Добавим значение площади под кривой
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc= roc_auc_score(target_valid, probabilities_one_valid)
print("Auc_roc:", auc_roc)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Глубина дерева: 13     Количество деревьев: 110   Качество: 0.8556338028169014 F1: 0.645021645021645
Auc_roc: 0.8540054671607091


## Тестирование модели

In [35]:
# Тестируем лучшую модель на тестовой выборке
model = RandomForestClassifier(max_depth=15, n_estimators=150, random_state=42)
model.fit(features_downsampled,target_downsampled)
predictions = model.predict(features_test)
print("Accuracy:", accuracy_score(predictions, target_test))
print("F1:", f1_score(predictions, target_test))

probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc= roc_auc_score(target_test, probabilities_one_valid)
print("Auc_roc:", auc_roc)

Accuracy: 0.8531222515391381
F1: 0.6033254156769596
Auc_roc: 0.8381466070786459


    Вывод: Модели с устраненным дисбалансом классов работают лучше. Лучший результат достугнут на случайном лесе при параметрах максимальная глубина дерева - 15 и число деревьев - 150

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*